Imports

In [ ]:
from google.colab import drive
import glob
import numpy as np
import pandas as pd
import os
from scipy.signal import stft
import scipy.io.wavfile as wav
from tqdm import tqdm
from collections import defaultdict

Sets directory and pulls files

In [ ]:
drive.mount('/content/drive', force_remount=True)
#change directory to ours
currentdir = os.getcwd()
print(currentdir)
if (currentdir != "/content/drive/.shortcut-targets-by-id/1YydIvgGXpybO4ZsE85TCIwj66s2vGuZJ/Cat Song Meter Recordings/FeliDetect Folder"):
  os.chdir("/content/drive/.shortcut-targets-by-id/1YydIvgGXpybO4ZsE85TCIwj66s2vGuZJ/Cat Song Meter Recordings/FeliDetect Folder")

file_list = glob.glob("*.wav",  recursive=True)
print(file_list)

Mounted at /content/drive
/content
['SMM07257_20221118_063302.wav']


converts seconds to timestamp

In [ ]:
def seconds_to_timestamp(seconds):
    """
    Converts seconds to a timestamp in HH:MM:SS format, with seconds rounded to two decimal places.
    """
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02}:{int(minutes):02}:{seconds:05.2f}"

finds events within magnitude and fequency threshold

In [ ]:
def find_events_within_threshold(file_path, dataset, min_mag=3500, max_mag=10000, min_freq=15, max_freq=300, segment_duration=.1, time_threshold=5):
    """
    Reads a WAV audio file, computes its STFT to find major magnitude events over time,
    and stores events that exceed a magnitude threshold.

    Parameters:
    - file_path (str): Path to the WAV file.
    - dataset (list): List of lists to store events ([file_path, start, end, magnitude, frequency, count]).
    - min_mag (float): The magnitude minimum for event detection (default is 3500).
    - max_mag (float): The magnitude maximum for event detection (default is 10000).
    - min_freq (float): The minimum frequency for event detection (default is 15hz).
    - max_freq (float): The maximum frequency for event detection (default is 300hz).
    - segment_duration (float): Duration of each STFT segment in seconds (default is 0.1s).
    - time_threshold (float): Time threshold in seconds for merging events (default is 5s).
    """
    try:
        sample_rate, audio_data = wav.read(file_path)
    except Exception as e:
        print(f"Error: {file_path} is not an audio file or doesn't exist. ({e})")
        return

    # If stereo, take just one channel
    if len(audio_data.shape) == 2:
        audio_data = audio_data[:, 0]

    # Convert audio data to np.float32 for efficiency
    audio_data = audio_data.astype(np.float32)
    # Remove DC offset by subtracting the mean
    audio_data -= np.mean(audio_data)

    # Compute the STFT with a specified segment duration
    nperseg = int(segment_duration * sample_rate)
    frequencies, times, Zxx = stft(audio_data, fs=sample_rate, nperseg=nperseg)
    magnitude = np.abs(Zxx)  # Magnitude of the STFT result

    # Initialize variables for event detection
    last_event_time_seconds = None
    event_data = []

    # Iterate over each time frame, using np.where to find indices where magnitude exceeds threshold
    for time_idx in range(magnitude.shape[1]):
        # Filter for magnitudes above the threshold
        magnitudes_at_time = magnitude[:, time_idx]
        valid_indices = np.where(np.logical_and(magnitudes_at_time < max_mag, magnitudes_at_time > min_mag))[0]

        if valid_indices.size == 0:
            continue  # Skip if no magnitudes exceed the threshold

        event_time_seconds = times[time_idx]
        event_time_str = seconds_to_timestamp(event_time_seconds)

        # Extract valid frequencies and magnitudes
        valid_frequencies = frequencies[valid_indices]
        valid_magnitudes = magnitudes_at_time[valid_indices]

        # Filter frequencies between 15 and 300 Hz
        freq_mask = (valid_frequencies < max_freq) & (valid_frequencies > min_freq)
        valid_frequencies = valid_frequencies[freq_mask]
        valid_magnitudes = valid_magnitudes[freq_mask]

        # Process events more efficiently
        for freq, mag in zip(valid_frequencies, valid_magnitudes):
            if last_event_time_seconds is None:
                # First event, add to dataset
                event_data.append([file_path, event_time_str, event_time_str, mag, freq, 1])
                last_event_time_seconds = event_time_seconds

            else:
                # Check if the event is within the 5-second window and more than .1 seconds later
                time_diff = event_time_seconds - last_event_time_seconds

                if time_diff <= time_threshold and time_diff > .1:  # Events within 5 seconds are merged
                    # Extend the duration of the last event
                    event_data[-1][2] = seconds_to_timestamp(event_time_seconds)
                    # increase count of calls
                    event_data[-1][5] += 1
                elif time_diff > time_threshold:
                    # Add new event
                    event_data.append([file_path, event_time_str, event_time_str, mag, freq, 1])
                # updates last event time
                last_event_time_seconds = event_time_seconds

    # Append all collected events to dataset at once
    dataset.extend(event_data)

counts calls per file

In [ ]:
def file_count(main_list, check_list):
    # Dictionary to store lists grouped by their first element
    groups = defaultdict(int)

    # Iterate through the main list
    for name in check_list:
        for sublist in main_list:
            if sublist[0] == name:
                groups[name] += sublist[5]
        if name not in groups:
          groups[name] = 0

    # Return a list of lists, each containing [file_name, count]
    return [[file_name, count] for file_name, count in groups.items()]


runs on all files

In [ ]:
    """
    - Parameters:
    - file_path (str): Path to the WAV file.
    - dataset (list): List of lists to store events ([file_path, start, end, magnitude, frequency, count]).
    - min_mag (float): The magnitude minimum for event detection (default is 3500).
    - max_mag (float): The magnitude maximum for event detection (default is 10000).
    - min_freq (float): The minimum frequency for event detection (default is 15hz).
    - max_freq (float): The maximum frequency for event detection (default is 300hz).
    - segment_duration (float): Duration of each STFT segment in seconds (default is 0.1s).
    - time_threshold (float): Time threshold in seconds for merging events (default is 5s).
    """
# modify these values to change the event
    min_mag = 3500
    max_mag = 10000
    min_freq = 15
    max_freq = 300
    segment_duration = .1
    time_threshold = 5

#adds progress bar
events = []
for file in tqdm(file_list, desc="Processing files", unit="file"):
     # extracts events
     find_events_within_threshold(file,events, min_mag, max_mag, min_freq, max_freq, segment_duration, time_threshold)
# counts saws per file
file_counts = file_count(events, file_list)

# saves results into an excel file
df = pd.DataFrame(events, columns=['File', 'Start', 'End', 'Magnitude', 'Frequency', 'Number of Calls'])

# Extract the date from the File column using string slicing and add it as a new column
df['Date'] = df['File'].str.extract(r'_(\d{8})_')[0]

#foward fill date
df['Date'] = df['Date'].ffill()

# Display the DataFrame with the new Date column
df.to_excel("events.xlsx")

df = pd.DataFrame(file_counts, columns=['File', 'Number of Events'])
df.to_excel("file_counts.xlsx")

Processing files:   0%|          | 0/1 [00:00<?, ?file/s]<ipython-input-10-6fa647a29839>:17: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio_data = wav.read(file_path)
Processing files: 100%|██████████| 1/1 [00:09<00:00,  9.24s/file]
